<a href="https://colab.research.google.com/github/PhilipMathieu/unet-orthoimagery/blob/PhilipMathieu%2Fissue6/colab_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# U-Net Training Notebook

In [1]:
!pip install wandb
!rm -r sample_data
!git clone --single-branch --branch PhilipMathieu/issue6 https://github.com/PhilipMathieu/unet-orthoimagery.git ./repo/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
rm: cannot remove 'sample_data': No such file or directory
fatal: destination path './repo' already exists and is not an empty directory.


[WandB Quickstart](https://docs.wandb.ai/quickstart#:~:text=Provide%20your%20API%20key%20when%20prompted.) (including link to your API key, if logged in)

In [2]:
import wandb
wandb.login()

wandb: Currently logged in as: philipmathieu (unet-ortho). Use `wandb login --relogin` to force relogin


True

In [3]:
import sys
sys.path.append('./repo/src/')

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import logging
import torch
from train import train_model
from unet.unet_model import UNet

/content/./repo/src/train.py:23: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [6]:
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f'Using device: {device}')
print(device)

cuda


In [7]:
model = UNet(n_channels=4, n_classes=1, bilinear=False)
model = model.to(memory_format=torch.channels_last)

In [8]:
logging.info(f'Network:\n'
                 f'\t{model.n_channels} input channels\n'
                 f'\t{model.n_classes} output channels (classes)\n'
                 f'\t{"Bilinear" if model.bilinear else "transposed conv"} upscaling')


model.to(device=device);

In [9]:
data_dir = "repo/data/Image_Chips_128_overlap_unbalanced_dem/"

In [10]:
def train():
    train_model(
        model=model,
        epochs=20,
        device=device,
        data_dir=data_dir,
        batch_size=128,
        img_scale=0.5,
        debug=False
    )

try:
    train()
except torch.cuda.OutOfMemoryError: # Giving me syntax error saying '"OutOfMemoryError" is not a valid exception class.'
    logging.error('Detected OutOfMemoryError! '
                    'Enabling checkpointing to reduce memory usage, but this slows down training. '
                    'Consider enabling AMP (--amp) for fast and memory efficient training')
    torch.cuda.empty_cache()
    model.use_checkpointing()
    train()

  0%|          | 0/654 [00:00<?, ?it/s]

  0%|          | 0/654 [00:00<?, ?it/s]

Epoch 1/20:   0%|          | 0/14358 [00:00<?, ?img/s]

Validation round:   0%|          | 0/56 [00:00<?, ?batch/s]

Validation round:   0%|          | 0/56 [00:00<?, ?batch/s]

...

Validation round:   0%|          | 0/56 [00:00<?, ?batch/s]

Validation round:   0%|          | 0/56 [00:00<?, ?batch/s]

Validation round:   0%|          | 0/56 [00:00<?, ?batch/s]

Epoch 3/20:   0%|          | 0/14358 [00:00<?, ?img/s]

KeyboardInterrupt: ignored

In [11]:
wandb.finish()

epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁████████████████████
learning rate,██████▁▁▁▁
step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train loss,█▅▃▂▁▂▂▁▃▃▂▃▁▂▁▃▂▃▂▃▃▂▃▁▃▂▂▁▁▂▂▂▂▂▂▂▁▂▃▁
validation Dice,▁▁▁▁▁▁▁▁▁▁
epoch,2
learning rate,1e-05
step,224
train loss,1.02341
validation Dice,0.9668


In [ ]:
!git -C repo pull